In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import networkx as nx

### for computing partial eigenvectors
import scipy.linalg as la
import scipy.sparse.linalg as sla

np.set_printoptions(precision=2, suppress=True)

In [ ]:
def spectral_embedding(g, d=2):
    n = g.order()
    L = nx.laplacian_matrix(g)
    if d < n:
        lam, Y = sla.eigsh(L.astype(float), k=d, which="SA")
    if d == n:
        lam, Y = la.eigh(L.toarray())
            
    return Y

def draw_graph(g, Y):
    n = g.order()
    fig = plt.figure()
    if Y.shape[1] == 2:
        ax = plt.axes()
    if Y.shape[1] == 3:
        ax = plt.axes(projection='3d')
    ### plot points
    ax.scatter(*Y.T, s=50, zorder=3)
    ### add vertex labels
    for i in range(n):
        ax.text(*Y[i], i, zorder=4)
    ### add lines
    for i,j in g.edges():
        d = g.get_edge_data(i,j)
        style = 'solid'
        if 'weight' in d.keys():
            if d['weight'] < 0.5:
                style = 'dashed'
        ax.plot(*Y[[i,j],:].T, 'c', linestyle=style)

In [ ]:
# random layout of 3K5
k5 = nx.complete_graph(5)
g1 = nx.disjoint_union_all([k5, k5, k5])
pos = nx.random_layout(g1, seed=2)
nx.draw_networkx(g1, pos=pos)

In [ ]:
# first three eigenvectors
Y = spectral_embedding(g1, 3)
Y

In [ ]:
draw_graph(g1, Y)

In [ ]:
# random layout of 3K5 + 2e
k5 = nx.complete_graph(5)
g2 = nx.disjoint_union_all([k5, k5, k5])
g2.add_edges_from([(4,5), (9,10)], weight=0.1)
# draw g2 with same pos as g1
pos = nx.random_layout(g1, seed=2)
elarge = g1.edges
esmall = [(4,5), (9,10)]
nx.draw_networkx_nodes(g2, pos)
nx.draw_networkx_labels(g2, pos)
nx.draw_networkx_edges(g2, pos, edgelist=elarge)
nx.draw_networkx_edges(g2, pos, edgelist=esmall, 
                       alpha=0.5, edge_color="b", style="dashed")

In [ ]:
# first three eigenvectors
Y = spectral_embedding(g2, 3)
Y

In [ ]:
draw_graph(g2, Y)

In [ ]:
# draw path graph
g = nx.path_graph(10)
Y = spectral_embedding(g, 3)
draw_graph(g, Y)

In [ ]:
# draw cycle graph
g = nx.cycle_graph(10)
Y = spectral_embedding(g, 3)
draw_graph(g, Y)

In [ ]:
# draw cube graph
g = nx.cubical_graph()
Y = spectral_embedding(g, 4)
draw_graph(g, Y[:,1:])

In [ ]:
# draw dodecahedron graph
g = nx.dodecahedral_graph()
Y = spectral_embedding(g, 4)
draw_graph(g, Y[:,1:])

In [ ]:
# download an image
import requests

URL = "https://github.com/jephianlin/LAwithNumPy/blob/main/incrediville-side.jpg?raw=true"
file = requests.get(URL, allow_redirects=True)

open("incrediville-side.jpg", "wb").write(file.content)

In [ ]:
from PIL import Image

img = Image.open("incrediville-side.jpg")
img = img.resize((img.size[0] // 80, img.size[1] // 80))
print(img.size)
img

In [ ]:
arr = np.array(img)
narr = arr / 255 # normalize 0 ~ 255
colorsT = narr.transpose(1,0,2).reshape(-1,3) # colors in RGB
m,n = arr.shape[:2]
mesh = np.meshgrid(np.arange(m), np.arange(n))
rind = mesh[0].ravel()
cind = mesh[1].ravel()
plt.scatter(cind, -rind, c=colorsT)

In [ ]:
grid = nx.grid_2d_graph(m,n)
colors = narr.reshape(-1,3) # colors in RGB
Y = spectral_embedding(grid, 3)
plt.scatter(*Y[:,1:].T, c=colors)

In [ ]:
grid = nx.grid_2d_graph(m,n)
colors = narr.reshape(-1,3) # colors in RGB
### set weights by similarity
for i,j in grid.edges:
    ij_dist = np.linalg.norm(narr[i] - narr[j])
    ij_sim = np.exp(-5*ij_dist**2)
    grid.edges[(i,j)]['weight'] = ij_sim

Y = spectral_embedding(grid, 3)
plt.scatter(*Y[:,1:].T, c=colors)